# Language Binding

The purpose of this notebook is for you to see as simple it is to call function or libraries from other programming languages. We show you few examples how to directly call and fully interoperate with Python and R from the Julia language, with the packages `PyCall` and `RCall`.


*References for this notebook:*
* PyCall Documentation: https://pkg.julialang.org/docs/PyCall/GkzkC/1.91.2/
* RCall Documentation: http://juliainterop.github.io/RCall.jl/stable/
* Excelling at Julia Basics and Beyond (Huda Nassar, Jane Herriman): 
    https://github.com/xorJane/Excelling-at-Julia-Basics-and-Beyond

# Outline of this notebook


- Calling python built-in function 
- Calling python hand-written function 
- Importing python libraries
- Calling R built-in function
- Calling R hand-written function

# Let choose an easy enough function to understand calling mechanisms: `sum`

Consider the  **sum** function `sum(x)`, which computes
$$
\mathrm{sum}(x) = \sum_{i=1}^n x_i,
$$
where $n$ is the length of `x`.

In [ ]:
# Generate avector of random numbers, uniform on [0,1)
a = rand(10^3);

    The expected result is 0.5 * 10^4, since the mean of each entry is 0.5

In [ ]:
 sum(a)

# 1. Calling Python's built-in `sum` 

The `PyCall` package provides a Julia interface to Python:

In [ ]:
# Using Pkg; Pkg.add("PyCall")
using PyCall

In [ ]:
# Get the Python built-in "sum" function:
pysum = pybuiltin("sum")

In [ ]:
pysum(a)

In [ ]:
# Confirm that the results of Julia and Python built in sum function are approximatly the same    
pysum(a) ≈ sum(a)

# 2. Calling Python hand-written `sum` function 

We can also write an easy Python code from within Julia.

### We can take an existing python function and wrap it into a julia function
_filename: python_sum.py_

```python
def py_sum(A):
    s = 0.0
    for a in A:
        s += a
    return s
```

In [ ]:
pushfirst!(PyVector(pyimport("sys")."path"), ""); #to tell pyimport search in the local directory

In [ ]:
python_sum = pyimport("python_sum") # wrote a function outside julia

In [ ]:
python_sum.py_sum(a)

### We can directly write the python function and wrap it into a julia function

In [ ]:
py"""
def py_sum(A):
    s = 0.0
    for a in A:
        s += a
    return s
"""
sum_py = py"py_sum"

In [ ]:
sum_py(a)

# 3. Importing Python libraries 

Here is a simple example to call Python `math` library and its sinus function `math.sin`:

In [ ]:
math = pyimport("math")
math.sin(math.pi / 4) 

## `numpy` takes advantage of  hardware "SIMD"

`numpy` is an optimized C library, callable from Python.
It may be installed within Julia as follows:

In [24]:
# using Pkg; Pkg.add("Conda")
using Conda
# In case numpy is not already installed run:
Conda.add("numpy")

┌ Info: Running `conda install -y numpy` in root environment
└ @ Conda C:\Users\gfarage\.julia\packages\Conda\kLXeC\src\Conda.jl:112


Solving environment: ...working... done

# All requested packages already installed.



In [ ]:
# We can use now the sum function from the `numpy` library
numpy_sum = pyimport("numpy")."sum"

In [ ]:
numpy_sum(a)

# 4. Calling R built-in `sum` 

Since it is true that Julia still lacks the depth and scale of the R package environment, let see how to interoperate R and Julia with `RCall`.

In [ ]:
# using Pkg; Pkg.add("RCall")
using RCall

We are able use directly base R function and pass arguments easily, as in the next example: 

In [ ]:
# Syntax Option 1
R"sum($a)"

In [ ]:
# Syntax Option 2
rcall(:sum, a)

In [ ]:
sum_R = convert(Float64, rcall(:sum, a))

# 5. Calling R hand-written function

### We can directly write the R function and wrap it into a julia function.

In [ ]:
R"""
library(vioplot); agrenViolinPlot <- function(){
agrenURL <- "https://bitbucket.org/linen/smalldata/raw/3c9bcd603b67a16d02c5dc23e7e3e637758d4d5f/arabidopsis/agren2013.csv"
agren <- read.csv(agrenURL); agrenFit <- agren[,c(1,2,3,4,5,6)]
vioplot(agrenFit, names=names(agrenFit), main = "ViolinPlot of Fitness Values per site and year", xlab ="Site", ylab =  "Fitness Values",col = rainbow(6))}
"""
R"agrenViolinPlot"();

### We can even use summary function from directly on our Julia dataframes

In [ ]:
using CSV, DataFrames
agrenURL = "https://bitbucket.org/linen/smalldata/raw/3c9bcd603b67a16d02c5dc23e7e3e637758d4d5f/arabidopsis/agren2013.csv"
agren = CSV.read(download(agrenURL),missingstring="NA");
 agren = dropmissing(agren, disallowmissing=true);

In [ ]:
rcall(:summary, agren)

# Summary

### Calling other languages makes Julia more productive and flexible.
### It is easy to call R and Python.
### Similar packages exists also for Java, C++ and C.